In [ ]:
import tkinter as tk                
from tkinter import font as tkfont
from tkinter import ttk
import mysql.connector
import tkinter.messagebox as m
from operator import itemgetter 
import re
from hashlib import md5
class Portal(tk.Tk):

    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self, *args, **kwargs)
        self.title("Supervisor Allocation Portal")
        self.geometry("800x500")
        self.title_font = tkfont.Font(family='Helvetica', size=18, weight="bold", slant="italic")

        # the container is where we'll stack a bunch of frames
        # on top of each other, then the one we want visible
        # will be raised above the others
        container = tk.Frame(self,padx=100,pady=20)
        container.pack(side="top", fill="both", expand=True)
        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)

        self.frames = {}
        for F in (Login_Page, Student_Details, Register):
            page_name = F.__name__
            frame = F(parent=container, controller=self)
            self.frames[page_name] = frame

            # put all of the pages in the same location;
            # the one on the top of the stacking order
            # will be the one that is visible.
            frame.grid(row=0, column=0, sticky="nsew")

        self.show_frame("Login_Page")

    def show_frame(self, page_name):
        '''Show a frame for the given page name'''
        frame = self.frames[page_name]
        frame.tkraise()
        
        
    def connect_db():
        global mydb                                         
        mydb = mysql.connector.connect(host='localhost',user='root',password='Pep@90210')
        mc = mydb.cursor()

        #creating database
        def create_db():
            try:                                          #error handling if databse is there
                mc.execute("create database DB")
            except:
                print("Database already created")
        create_db()

        #using database
        mc.execute("use DB")

        #creating table
        def create_table():
            try:                                         #error handling if table is there
                mc.execute("create table detail(name varchar(63),reg_no varchar(12),email_id varchar(63),spec varchar(63),password varchar(63))")
            except:
                print("Table already created")
        create_table()
    connect_db()
   
    

class Login_Page(tk.Frame):

    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        self.controller = controller
        
        reg_no,password=tk.StringVar(),tk.StringVar()
        
        def clearEntryBox():
            Pass_entry.delete(first=0,last=100)
            Regno_entry.delete(first=0,last=100)
        
        def login(): 

            if(reg_no.get()=="" or password.get()==""):
                m.showerror(title = "Error", message = "Please Enter the Regtration ID and Password")
            
            elif(not(reg_no.get().isdigit()) or not(len(reg_no.get())==8)):
                m.showerror(title = "Error", message = "Please Enter Valid Registraion Number")
                
            else:
                check()
            
            
        def register():
            clearEntryBox()
            controller.show_frame("Register")
            
            
            
        def check(): 
            cursor_1 = mydb.cursor(buffered=True)
            cursor_2 = mydb.cursor(buffered=True)
            sqlCommand1 = "select reg_no from detail"
            sqlCommand2 = "select password from detail"
            cursor_1.execute(sqlCommand1)
            cursor_2.execute(sqlCommand2)
            
            p= []
            e = []
            for i in cursor_1:
                e.append(i)
                
            for j in cursor_2:
                p.append(j)
            res1 = list(map(itemgetter(0), e))
            res2 = list(map(itemgetter(0), p))
            k=len(res2)
            i=0
            h_password=(md5(password.get().encode())).hexdigest()
            while i<k:
                if res1[i]==reg_no.get() and res2[i]==h_password:
                    mydb.commit()
                    clearEntryBox()
                    controller.show_frame("Student_Details")
                    break
                i+=1
            if(i==k):
                mydb.commit()
                m.showerror("Error","Incorrect Regtration ID and Password")    
            
            
        
        Welcome=tk.Label(self,text='LPU Capstone Supervisor Allocation Portal',font='comicsansms 20 bold',pady=15)
        Welcome.grid(row=0,column=4)

        Regno_lable=tk.Label(self,text='Regtration ID',padx=5)
        Regno_lable.grid(row=1,column=2)
        Regno_entry=tk.Entry(self,bd=4,width=30,textvariable=reg_no)
        Regno_entry.grid(row=1,column=3,columnspan=3)
        Regno_entry.focus()

        Pass_lable=tk.Label(self,text='Password',padx=5)
        Pass_lable.grid(row=3,column=2)
        Pass_entry=tk.Entry(self,bd=4,width=30,show="*",textvariable=password)
        Pass_entry.grid(row=3,column=3,columnspan=3)

        Login_bt=tk.Button(self,text="Login",command=login,fg='green')
        Login_bt.grid(row=5,column=4)

        Reg_Message=tk.Label(self,text="Have'nt Regester Yet. Click on Register Button.",font="Verdana 12 bold",fg='red')
        Reg_Message.grid(row=7,column=3,columnspan=4)

        Reg_bt=tk.Button(self,text="Register",command=register,fg='green')
        Reg_bt.grid(row=9,column=4)
        
        
        
class Student_Details(tk.Frame):

    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        self.controller = controller
        
        def Homepage():
            controller.show_frame("Login_Page")
        
        Welcome=tk.Label(self,text='Welcome',font='Verdana 12 bold',padx=15)
        Welcome.place(x=10,y=30)
        
        Message=tk.Label(self,text='Your Capstone Supervisor will be appointed soon',font='comicsansms 20 bold',fg='red',padx=15)
        Message.place(x=40,y=80)
        
        Home_bt=tk.Button(self,text="Back to Home Page",command=Homepage,fg='green')
        Home_bt.place(x=210,y=140)
        
        
        
class Register(tk.Frame):

    def __init__(self, parent, controller):
        tk.Frame.__init__(self, parent)
        self.controller = controller
        
        name,reg_no,email_id,spec,password,repassword=tk.StringVar(),tk.StringVar(),tk.StringVar(),tk.StringVar(),tk.StringVar(),tk.StringVar()
        
        def clearEntryBox():
            Name_entry.delete(first=0,last=100)
            Regno_entry.delete(first=0,last=100)
            Email_entry.delete(first=0,last=100)
            Option.set("")
            Pass_entry.delete(first=0,last=100)
            Re_Pass_entry.delete(first=0,last=100)
    
        def Homepage():
            clearEntryBox()
            controller.show_frame("Login_Page")
        
        
        def insert():

            if(name.get()=="" or reg_no.get()=="" or email_id.get()=="" or spec.get()=="" or password.get()==""or repassword.get()==""):
                m.showerror(title = "Error", message = "Please Enter the Details in all Fields")

            elif(not(reg_no.get().isdigit()) or not(len(reg_no.get())==8)):
                m.showerror(title = "Error", message = "Please Enter a Valid Registraion Number")

            elif(not(bool(re.match('[a-zA-Z\s]+$',name.get())))):
                 m.showerror(title = "Error", message = "Please Enter a Valid Name")

            elif(not(password.get() == repassword.get())):
                 m.showerror(title = "Error", message = "Passwords do not match")
            
            elif(len(password.get())<8):
                 m.showerror(title = "Error", message = "Please Enter a Strong Password")
                    
            else:
                cursor2 = mydb.cursor()
                sqlCommand1 = "select reg_no from detail"
                cursor2.execute(sqlCommand1)
                
                p=[]
                for i in cursor2:
                    p.append(i)
                    
                res = list(map(itemgetter(0), p))
                regno=reg_no.get()
                k=len(res)
                i,flag=0,0
                while i <k:
                    if res[i]==reg_no.get():
                        m.showerror("Error","Student with Entered Registration ID was Already Registered")
                        flag=1
                        break
                    i+=1
                mydb.commit()
              
                if(flag==0):
                    cursor1 = mydb.cursor()
                    h_password=(md5(password.get().encode())).hexdigest()
                    sqlCommand = ("insert into detail (name,reg_no,email_id,spec,password) values(%s,%s,%s,%s,%s)")
                    values = (name.get(),reg_no.get(),email_id.get(),spec.get(),h_password)
                    cursor1.execute(sqlCommand,values)
                    mydb.commit()
                    clearEntryBox()
                    m.showinfo(title = "Confirmation", message = "Registration Successful")

        
        Welcome=tk.Label(self,text='Enter Your Details',font='comicsansms 20 bold',padx=15,fg='red')
        Welcome.place(x=220,y=0)
        
        Name_lable=tk.Label(self,text='Full Name',padx=5)
        Name_lable.place(x=40,y=50)
        Name_entry=tk.Entry(self,bd=4,width=30,textvariable=name)
        Name_entry.focus()
        Name_entry.place(x=180,y=50)
        
        
        Regno_lable=tk.Label(self,text='Regtration ID',padx=5)
        Regno_lable.place(x=30,y=90)
        Regno_entry=tk.Entry(self,bd=4,width=30,textvariable=reg_no)
        Regno_entry.place(x=180,y=90)
        
        Email_lable=tk.Label(self,text='Email ID',padx=5)
        Email_lable.place(x=45,y=130)
        Email_entry=tk.Entry(self,bd=4,width=30,textvariable=email_id)
        Email_entry.place(x=180,y=130)
        
        Message_Sp = tk.Label(self,text = "Select your Specialization",pady=5)
        Message_Sp.place(x=5,y=170)

        Option = ttk.Combobox(self,state="readonly",values=["CSE","ECE","IT","MEC","CVL"],textvariable=spec)
        Option.place(x=183,y=170)

        Pass_lable=tk.Label(self,text='Password',padx=5)
        Pass_lable.place(x=38,y=210)
        Pass_entry=tk.Entry(self,bd=4,width=30,show="*",textvariable=password)
        Pass_entry.place(x=180,y=210)
        
        Re_Pass_lable=tk.Label(self,text='Confirm Password',padx=5)
        Re_Pass_lable.place(x=10,y=250)
        Re_Pass_entry=tk.Entry(self,bd=4,width=30,show="*",textvariable=repassword)
        Re_Pass_entry.place(x=180,y=250)
        
        
        Reg_bt=tk.Button(self,text="Submit",command=insert,fg='red')
        Reg_bt.place(x=270,y=300)
        
        Home_bt=tk.Button(self,text="Back to Home Page",command=Homepage,fg='green')
        Home_bt.place(x=232,y=340)
                 
                        
app = Portal()
app.mainloop()

